# WorkSafe and Motor Vehicle audit report

In [2]:
import os
import psycopg2
import pandas as pd
import csv
import matplotlib
from datetime import datetime, timedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL



In [19]:
connect_to_db = 'postgresql://' + \
                os.getenv('DB_USER', '') + ":" + os.getenv('DB_PASS', '') +'@' + \
                os.getenv('DB_HOST', '') + ':' + os.getenv('DB_PORT', '5432') + '/' + os.getenv('DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working



In [20]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

In [21]:
%%sql worksafe_corp_audit  <<
select b.identifier,f.filing_type,f.transaction_id,f.filing_date at time zone 'America/Vancouver'
from filings    f
    ,businesses b
where f.business_id=b.id
  and b.identifier in ('BC1381431', 'S0044626', 'S0034315','BC0786328','BC0455835','FM0329275','FM0326476','FM0165333','FM0232910','FM0556598','BC0443045'
                      ,'BC0812666','BC0613715', 'S0037540','BC0047143','BC0937794','BC1182816','FM0753730','BC1118811','BC1196208','BC0842217','BC0559626'
                      ,'BC1162353','BC0961510','BC0282129','BC1307937', 'A0120144','BC0527597','BC0842217','BC1155160')
  and to_char(filing_date at time zone 'America/Vancouver','yyyymmdd') = to_char(now() at time zone 'America/Vancouver' - interval '1 day','yyyymmdd')


In [1]:
corp_audit = worksafe_corp_audit.DataFrame()

datestr = datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d')
filename =  os.path.join(os.getcwd(), r'data/')+'worksafe_audit_' + datestr +'.csv'

with open(filename, 'w') as f:
    if corp_audit.empty:
        f.write('No Data Retrieved for any corp of ineterest ' + datestr)         
    else:  
        f.write('WorkSafe: Audit of businesses for malicious activity ' + datestr + ':\n')            
        corp_audit.to_csv(f, sep=',', encoding='utf-8', index=False)  

In [22]:
%%sql worksafe_name_audit  <<

select b.identifier,f.filing_type,f.transaction_id,f.filing_date at time zone 'America/Vancouver',p.first_name,p.last_name
from parties_version p
    ,filings         f
    ,businesses      b
where
(
   (upper(last_name) = 'GALLO'    and  upper(first_name) like 'AN%')
or (upper(last_name) = 'GALLO'    and  upper(first_name) like 'DE%')
or (upper(last_name) = 'BLOOR'    and  upper(first_name) like 'BR%')
or (upper(last_name) = 'HOLLER'   and (upper(first_name) like 'AN%' or  upper(first_name) = 'TONY'))
or (upper(last_name) = 'MCMILLAN' and  upper(first_name) like 'ST%')
or (upper(last_name) = 'LABONTE'  and  upper(first_name) like 'NO%')
or (upper(last_name) = 'THOMPSON' and  upper(first_name) like 'AM%')
or (upper(last_name) = 'SWANSON'  and  upper(first_name) like 'LI%')
or (upper(last_name) = 'TRONSON'  and  upper(first_name) like 'MA%')
or (upper(last_name) = 'MYERS'    and  upper(first_name) like 'SU%')
)
and p.transaction_id=f.transaction_id
and f.business_id=b.id
and to_char(filing_date at time zone 'America/Vancouver','yyyymmdd') = to_char(now() at time zone 'America/Vancouver' - interval '1 day','yyyymmdd')

In [ ]:
name_audit = worksafe_name_audit.DataFrame()

with open(filename, 'a') as f:  
    if name_audit.empty:
        f.write('No Data Retrieved for Names audit ' + datestr)         
    else: 
        f.write('WorkSafe: Audit of people for malicious activity ' + datestr + ':\n')  
        name_audit.to_csv(f, sep=',', encoding='utf-8', index=False) 
    f.write('\n\n')  

In [23]:
%%sql worksafe_name_audit2  <<

select username,firstname,lastname,transaction_id,creation_date at time zone 'America/Vancouver'
from users_version u
where
(
   (upper(lastname) = 'GALLO'    and  upper(firstname) like 'AN%')
or (upper(lastname) = 'GALLO'    and  upper(firstname) like 'DE%')
or (upper(lastname) = 'BLOOR'    and  upper(firstname) like 'BR%')
or (upper(lastname) = 'HOLLER'   and (upper(firstname) like 'AN%' or  upper(firstname) = 'TONY'))
or (upper(lastname) = 'MCMILLAN' and  upper(firstname) like 'ST%')
or (upper(lastname) = 'LABONTE'  and  upper(firstname) like 'NO%')
or (upper(lastname) = 'THOMPSON' and  upper(firstname) like 'AM%')
or (upper(lastname) = 'SWANSON'  and  upper(firstname) like 'LI%')
or (upper(lastname) = 'TRONSON'  and  upper(firstname) like 'MA%')
or (upper(lastname) = 'MYERS'    and  upper(firstname) like 'SU%')
)
and to_char(creation_date at time zone 'America/Vancouver','yyyymmdd') = to_char(now() at time zone 'America/Vancouver' - interval '1 day','yyyymmdd')

In [ ]:
name_audit2 = worksafe_name_audit2.DataFrame()

with open(filename, 'a') as f:  
    if name_audit2.empty:
        f.write('No Data Retrieved for Names audit ' + datestr)         
    else: 
        f.write('WorkSafe: Audit of people for malicious activity ' + datestr + ':\n')  
        name_audit2.to_csv(f, sep=',', encoding='utf-8', index=False) 
    f.write('\n\n')  

In [24]:
%%sql motor_vehicle_audit  <<

select b.identifier,f.filing_type,f.transaction_id,f.filing_date at time zone 'America/Vancouver'
from filings    f
    ,businesses b
where f.business_id=b.id
  and b.identifier = 'BC1367734'
  and to_char(filing_date at time zone 'America/Vancouver','yyyymmdd') = to_char(now() at time zone 'America/Vancouver' - interval '1 day','yyyymmdd')

In [ ]:
motor_vehicle_audit = motor_vehicle_audit.DataFrame()

with open(filename, 'a') as f:  
    if motor_vehicle_audit.empty:
        f.write('No Data Retrieved for motor vehicle audit ' + datestr)         
    else: 
        f.write('Motor Vehicle: Audit of businesses for malicious activity ' + datestr + ':\n')  
        motor_vehicle_audit.to_csv(f, sep=',', encoding='utf-8', index=False) 
    f.write('\n\n')  

In [25]:
%%sql motor_vehicle_name_audit1  <<

select b.identifier,f.filing_type,f.transaction_id,f.filing_date at time zone 'America/Vancouver',p.first_name,p.last_name
from parties_version p
    ,filings         f
    ,businesses      b
where
(
   (upper(last_name) = 'TEPERTO'  and  upper(first_name) like 'MI%')
or (upper(last_name) = 'THOMSON'  and  upper(first_name) like 'CA%')
)
and p.transaction_id=f.transaction_id
and f.business_id=b.id
and to_char(filing_date at time zone 'America/Vancouver','yyyymmdd') = to_char(now() at time zone 'America/Vancouver' - interval '1 day','yyyymmdd')

In [ ]:
motor_vehicle_name_audit1 = motor_vehicle_name_audit1.DataFrame()

with open(filename, 'a') as f:  
    if motor_vehicle_name_audit1.empty:
        f.write('No Data Retrieved for motor vehicle name audit ' + datestr)         
    else: 
        f.write('Motor Vehicle: Audit of people for malicious activity ' + datestr + ':\n')  
        motor_vehicle_name_audit1.to_csv(f, sep=',', encoding='utf-8', index=False) 
    f.write('\n\n')  

In [26]:
%%sql motor_vehicle_name_audit2  <<

select username,firstname,lastname,transaction_id,creation_date at time zone 'America/Vancouver'
from users_version u
where
(
   (upper(lastname) = 'TEPERTO'  and  upper(firstname) like 'MI%')
or (upper(lastname) = 'THOMSON'  and  upper(firstname) like 'CA%')
)
and to_char(creation_date at time zone 'America/Vancouver','yyyymmdd') = to_char(now() at time zone 'America/Vancouver' - interval '1 day','yyyymmdd')

In [ ]:
motor_vehicle_name_audit2 = motor_vehicle_name_audit2.DataFrame()

with open(filename, 'a') as f:  
    if motor_vehicle_name_audit2.empty:
        f.write('No Data Retrieved for motor vehicle name audit ' + datestr)         
    else: 
        f.write('Motor Vehicle: Audit of people for malicious activity ' + datestr + ':\n')  
        motor_vehicle_name_audit2.to_csv(f, sep=',', encoding='utf-8', index=False) 
    f.write('\n\n')  

In [27]:
%%sql rcmp_audit  <<

select b.identifier,f.filing_type,f.transaction_id,f.filing_date at time zone 'America/Vancouver'
from filings    f
    ,businesses b
where f.business_id=b.id
  and b.identifier = 'BC1374478'
  and to_char(filing_date at time zone 'America/Vancouver','yyyymmdd') = to_char(now() at time zone 'America/Vancouver' - interval '1 day','yyyymmdd')

In [ ]:
rcmp_audit = rcmp_audit.DataFrame()

with open(filename, 'a') as f:  
    if rcmp_audit.empty:
        f.write('No Data Retrieved for RCMP audit ' + datestr)         
    else: 
        f.write('Richmond RCMP: Audit of businesses for malicious activity (file 2023-1790) ' + datestr + ':\n')  
        rcmp_audit.to_csv(f, sep=',', encoding='utf-8', index=False) 
    f.write('\n\n')  